In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import multiprocessing

In [2]:
cis_df_all_eQTL_sig = pd.read_csv('/CIMA/Result/eQTL_L1_downstream/20250219_cis_eQTL_studywise_sig.csv',index_col=0)

# 处理onek1k数据

In [3]:
public_eQTL = pd.read_csv('/CIMA/Data/public_eQTL/20250218_file_38_for_comparasion/20250218_onek1k_leap_snp.csv',index_col=0)
#首先，GENE得在我们显著的eGene里面，其次，这个位点可以不在我们的检测里面（统计数量，这就是频率差异），然后，这个位点不同的在去做分析
public_eQTL = public_eQTL[public_eQTL['GENE'].isin(cis_df_all_eQTL_sig['phenotype_id'].unique())]

# onek1k中 eGene属于CIMA L1eGene有9000个

In [4]:
len(public_eQTL)

9094

In [5]:
variant_df = pd.read_parquet('/CIMA/Data/413_sample_variant_df.parquet')

public_eQTL.loc[:,'variant_id'] = public_eQTL['CHR']+'_'+public_eQTL['POS'].astype(str)

public_eQTL = public_eQTL[public_eQTL['variant_id'].isin(variant_df.index)]

# onek1k中 eGene属于CIMA L1以及位点被检测的有6000个

In [6]:
len(public_eQTL)

6313

In [7]:
# 创建一个空的DataFrame来存储比例结果
overlap_count = pd.DataFrame(index=cis_df_all_eQTL_sig['celltype'].unique(), columns=['overlap_eGene','overlap_eGene_with_diffsnp'])

In [8]:
# 创建一个空的DataFrame来存储比例结果
overlap_count = pd.DataFrame(index=cis_df_all_eQTL_sig['celltype'].unique(), columns=['overlap_full','overlap_with_diff_topSNP'])

df_merge = pd.DataFrame()

for celltype in ['CD4_T','CD8_T','NK','Myeloid','B']:
    use_df_A = cis_df_all_eQTL_sig[cis_df_all_eQTL_sig['celltype'] == celltype][['phenotype_id','variant_id']]
    series_A = pd.Series(use_df_A['variant_id'].values, index=use_df_A['phenotype_id'].values)
    use_df_B =public_eQTL[public_eQTL['celltype_map'] == celltype][['GENE','variant_id']]
    series_B = pd.Series(use_df_B['variant_id'].values, index=use_df_B['GENE'].values)
    # 合并两个 Series，保留共享的索引，并设置列名
    df = pd.concat([series_A, series_B], axis=1, join='inner').reset_index()
    df.columns = ['phenotype','topSNP_CIMA', 'topSNP_public']
    df['celltype'] = celltype
    df = df[['celltype','phenotype','topSNP_CIMA', 'topSNP_public']]

    # 统计合并后的行数
    initial_row_count = len(df)
    overlap_count.loc[celltype,'overlap_full'] = initial_row_count

    # 去除 topSNP_A 等于 topSNP_B 的行
    df_filtered = df[df['topSNP_CIMA'] != df['topSNP_public']]

    # 统计去除后的行数
    filtered_row_count = len(df_filtered)
    overlap_count.loc[celltype,'overlap_with_diff_topSNP'] = filtered_row_count
    df_merge = pd.concat([df_merge,df_filtered]).reset_index(drop = True)

In [9]:
df_merge.to_csv(f'/CIMA/Data/public_independent/20250220_onk1k_independent_input.csv')
overlap_count.to_csv(f'/CIMA/Result/eQTL_L1_downstream/20250220_onk1k_each_celltype_overlap.csv')

In [10]:
#输出所有显著eSNP的基因分型
genotype_df = pd.read_parquet('/CIMA/Data/413_sample_genotype.parquet')

combined_list = list(df_merge['topSNP_CIMA']) + list(df_merge['topSNP_public'])

genotype_df.loc[np.unique(combined_list),:].T.to_csv('/CIMA/Data/public_independent/genotype_sigeSNP_onek1k.csv')

# 处理immuenexut数据

In [11]:
public_eQTL = pd.read_csv('/CIMA/Data/public_eQTL/20250218_file_38_for_comparasion/20250219_immuenexut_all_lead_snp.csv',index_col=0)
#首先，GENE得在我们显著的eGene里面，其次，这个位点可以不在我们的检测里面（统计数量，这就是频率差异），然后，这个位点不同的在去做分析
public_eQTL = public_eQTL[public_eQTL['Gene_name'].isin(cis_df_all_eQTL_sig['phenotype_id'].unique())]

In [12]:
len(public_eQTL)

37886

In [13]:
variant_df = pd.read_parquet('/CIMA/Data/413_sample_variant_df.parquet')
public_eQTL.loc[:,'variant_id'] = public_eQTL['Variant_CHR']+'_'+public_eQTL['Variant_position_start'].astype(str)
public_eQTL = public_eQTL[public_eQTL['variant_id'].isin(variant_df.index)]

In [14]:
len(public_eQTL)

26749

In [15]:
# 创建一个空的DataFrame来存储比例结果
overlap_count = pd.DataFrame(index=cis_df_all_eQTL_sig['celltype'].unique(), columns=['overlap_full','overlap_with_diff_topSNP'])

df_merge = pd.DataFrame()

for celltype in ['CD4_T','CD8_T','NK','Myeloid','B']:
    use_df_A = cis_df_all_eQTL_sig[cis_df_all_eQTL_sig['celltype'] == celltype][['phenotype_id','variant_id']]
    series_A = pd.Series(use_df_A['variant_id'].values, index=use_df_A['phenotype_id'].values)
    use_df_B =public_eQTL[public_eQTL['celltype_map'] == celltype][['Gene_name','variant_id']]
    series_B = pd.Series(use_df_B['variant_id'].values, index=use_df_B['Gene_name'].values)
    # 合并两个 Series，保留共享的索引，并设置列名
    df = pd.concat([series_A, series_B], axis=1, join='inner').reset_index()
    df.columns = ['phenotype','topSNP_CIMA', 'topSNP_public']
    df['celltype'] = celltype
    df = df[['celltype','phenotype','topSNP_CIMA', 'topSNP_public']]

    # 统计合并后的行数
    initial_row_count = len(df)
    overlap_count.loc[celltype,'overlap_full'] = initial_row_count

    # 去除 topSNP_A 等于 topSNP_B 的行
    df_filtered = df[df['topSNP_CIMA'] != df['topSNP_public']]

    # 统计去除后的行数
    filtered_row_count = len(df_filtered)
    overlap_count.loc[celltype,'overlap_with_diff_topSNP'] = filtered_row_count
    df_merge = pd.concat([df_merge,df_filtered]).reset_index(drop = True)

In [16]:
df_merge.to_csv(f'/CIMA/Data/public_independent/20250220_immuenexut_independent_input.csv')
overlap_count.to_csv(f'/CIMA/Result/eQTL_L1_downstream/20250220_immuenexut_each_celltype_overlap.csv')

In [17]:
#输出所有显著eSNP的基因分型
genotype_df = pd.read_parquet('/CIMA/Data/413_sample_genotype.parquet')
combined_list = list(df_merge['topSNP_CIMA']) + list(df_merge['topSNP_public'])
genotype_df.loc[np.unique(combined_list),:].T.to_csv('/CIMA/Data/public_independent/genotype_sigeSNP_immuenexut.csv')